In [2]:
!pip install openpyxl 
import pandas as pd

df = pd.ExcelFile('exchange_rate_manual_trade_r2.xlsx')

sheet_names = df.sheet_names
sheet_names

Defaulting to user installation because normal site-packages is not writeable


['Sheet1']

In [3]:
exchange_rates = df.parse(sheet_names[0])
exchange_rates

,Unnamed: 0,Pizza slice,Wasabi root,Snowball,Shells
0,Pizza slice,1.00,0.48,1.52,0.71
1,Wasabi root,2.05,1.00,3.26,1.56
2,Snowball,0.64,0.30,1.00,0.46
3,Shells,1.41,0.61,2.08,1.00


In [13]:
# Extract rates into a dictionary for easier processing
currencies = exchange_rates.columns[1:].tolist()  # Excluding the 'Unnamed: 0' which holds the row headers
rates_dict = {row['Unnamed: 0']: row[1:].to_dict() for index, row in exchange_rates.iterrows()}

# Let's perform some calculations to find the best trade path
def find_best_trade(start_currency, trades_left, current_amount=1, path=[]):
    if trades_left == 0:
        # We must end with Shells
        if start_currency != 'Shells':
            return 0, []
        else:
            return current_amount, path
    
    best_amount = 0
    best_path = []
    
    for target_currency in currencies:
        if target_currency == start_currency:
            continue  # Skip trading to the same currency
        new_amount = current_amount * rates_dict[start_currency][target_currency]
        new_path = path + [(start_currency, target_currency, new_amount)]
        result_amount, result_path = find_best_trade(target_currency, trades_left - 1, new_amount, new_path)
        if result_amount > best_amount:
            best_amount = result_amount
            best_path = result_path

    return best_amount, best_path

# Start trading from Shells with 5 trades available
max_seashells, optimal_trade_path = find_best_trade('Shells', 5)

max_seashells, optimal_trade_path


(1.0569693888,
 [('Shells', 'Pizza slice', 1.41),
  ('Pizza slice', 'Wasabi root', 0.6768),
  ('Wasabi root', 'Shells', 1.055808),
  ('Shells', 'Pizza slice', 1.48868928),
  ('Pizza slice', 'Shells', 1.0569693888)])

In [14]:
# Initial amount of SeaShells
initial_seashells = 2000000

# Calculate the final amount of SeaShells after completing the optimal trades
final_seashells = initial_seashells * max_seashells
return_rate = final_seashells / initial_seashells
final_seashells, return_rate

(2113938.7776, 1.0569693888)